In [1]:
import os,sys,datetime
import numpy as np
from scipy.spatial.distance import *
from copy import deepcopy
from hybrid_mdmc.data_file_parser import parse_data_file
from hybrid_mdmc.lammps_files_classes import LammpsInitHandler
from hybrid_mdmc.customargparse import HMDMC_ArgumentParser
from hybrid_mdmc.classes import *
from hybrid_mdmc.parsers import *
from hybrid_mdmc.kmc import *
from hybrid_mdmc.functions import *
from hybrid_mdmc.calc_voxels import *
from hybrid_mdmc.diffusion import *
from textwrap import dedent

In [9]:
notebook = pd.read_excel('toy8_notebook.xlsx',sheet_name=None,index_col=None,header=None)
notebook['Species'] = notebook['Species'].astype({col:object for col in notebook['Species'].columns})
notebook['Species'].replace(['nan', '-'], np.nan, inplace=True)#.infer_objects()
notebook['Species'].dropna(how='all', inplace=True)

print(notebook['Species'])
print(notebook['Species'].dtypes)

                               0       1    2     3     4    5
0                        Species     NaN    A    A2   NaN    B
2   Starting Number of Particles     NaN  200   100   NaN  600
4                          Atoms      ID    1     1  2.00    1
5                            NaN    type    1     1  1.00    2
6                            NaN  charge    0     0  0.00    0
7                            NaN       x    0 -0.75  0.75    0
8                            NaN       y    0     0  0.00    0
9                            NaN       z    0     0  0.00    0
11                         Bonds      ID  NaN     1   NaN  NaN
12                           NaN    type  NaN     1   NaN  NaN
13                           NaN       i  NaN     1   NaN  NaN
14                           NaN       j  NaN     2   NaN  NaN
16                        Angles      ID  NaN   NaN   NaN  NaN
17                           NaN    type  NaN   NaN   NaN  NaN
18                           NaN       i  NaN   NaN   N

In [3]:
type(np.nan)

float